# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels]

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels]

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [5]:
input_dim = 122
intermediate_dim = 122
latent_dim = 32
batch_size = 1409
epochs = 5
hidden_layers = 8

class Train:
    def train():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu')(hidden_encoder)

        Train.mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)
        Train.logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            m_e, l_e = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = m_e + K.exp(l_e / 2) * epsilon
            return z

        z = Lambda(get_distrib)([Train.mean_encoder, Train.logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu")(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None)(hidden_decoder)

def vae_loss(x, x_decoded_mean):
    xent_loss = input_dim * keras.losses.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + Train.logvar_encoder - K.square(Train.mean_encoder) - K.exp(Train.logvar_encoder), axis=-1)
    return K.mean(xent_loss + kl_loss)


In [6]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

features_arr = [4, 16, 32]
hidden_layers_arr = [2, 6, 10]

epoch_arr = [50]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = pd.DataFrame()

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h

    Train.train()

    optimizer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-04, decay=0.1)
    vae_model = Model(inputs = Train.x, outputs = Train.x_ )
    vae_model.compile(optimizer = optimizer, loss = vae_loss, metrics = ['accuracy'] )

    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    vae_model.fit(x = x_train[:train_size,:], y = x_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  validation_data = (x_test, x_test),
                  verbose = 1)
    
    score_train = vae_model.evaluate(x_valid[:valid_size,:], y = x_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = vae_model.evaluate(x_test, y = x_test,
                           batch_size = batch_size,
                           verbose = 1)
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)


 
 Current Layer Attributes - epochs:50 hidden layers:2 features count:4
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 1s - loss: 59185098081499.0703 - acc: 0.0039 - val_loss: 1293461441478684.2500 - val_acc: 0.0065
Epoch 2/50
112720/112720 [==============================] - 1s - loss: 7139495217530.3066 - acc: 0.0045 - val_loss: 1054745443696664.8750 - val_acc: 0.0065
Epoch 3/50
112720/112720 [==============================] - 1s - loss: 5910421213208.3340 - acc: 0.0044 - val_loss: 937611831017494.8750 - val_acc: 0.0068
Epoch 4/50
112720/112720 [==============================] - 1s - loss: 5206723521225.0312 - acc: 0.0045 - val_loss: 839572898447381.2500 - val_acc: 0.0075
Epoch 5/50
112720/112720 [==============================] - 1s - loss: 4700238987452.2910 - acc: 0.0045 - val_loss: 828538521387028.3750 - val_acc: 0.0083
Epoch 6/50
112720/112720 [==============================] - 1s - loss: 4398166007016.8057 - acc: 0

112720/112720 [==============================] - 2s - loss: 131927236674.1404 - acc: 7.9844e-05 - val_loss: 699517099851.9182 - val_acc: 0.0049
Epoch 2/50
112720/112720 [==============================] - 1s - loss: 25218673613.5140 - acc: 8.8715e-05 - val_loss: 503023790820.5268 - val_acc: 0.0050
Epoch 3/50
112720/112720 [==============================] - 1s - loss: 19510970321.0946 - acc: 8.8715e-05 - val_loss: 456706652854.4053 - val_acc: 0.0050
Epoch 4/50
112720/112720 [==============================] - 1s - loss: 16985041480.1480 - acc: 7.9844e-05 - val_loss: 397499296909.3220 - val_acc: 0.0047
Epoch 5/50
112720/112720 [==============================] - 1s - loss: 15258304301.0297 - acc: 9.7587e-05 - val_loss: 359543339628.7512 - val_acc: 0.0047
Epoch 6/50
112720/112720 [==============================] - 1s - loss: 13916589795.5411 - acc: 9.7587e-05 - val_loss: 336031707731.8146 - val_acc: 0.0046
Epoch 7/50
112720/112720 [==============================] - 1s - loss: 12921181279.917

112720/112720 [==============================] - 1s - loss: 205802160593003.4688 - acc: 0.0062 - val_loss: 74805208354156.4531 - val_acc: 0.0017
Epoch 3/50
112720/112720 [==============================] - 1s - loss: 136967922815431.3750 - acc: 0.0062 - val_loss: 64040250392906.9453 - val_acc: 0.0021
Epoch 4/50
112720/112720 [==============================] - 1s - loss: 118285809503729.6719 - acc: 0.0064 - val_loss: 57410136747317.6328 - val_acc: 0.0021
Epoch 5/50
112720/112720 [==============================] - 1s - loss: 105972665301211.1719 - acc: 0.0064 - val_loss: 55255236500214.8281 - val_acc: 0.0022
Epoch 6/50
112720/112720 [==============================] - 1s - loss: 97676459714987.0938 - acc: 0.0065 - val_loss: 49411210618264.3359 - val_acc: 0.0022
Epoch 7/50
112720/112720 [==============================] - 1s - loss: 90320603471223.8750 - acc: 0.0065 - val_loss: 46658266763076.4141 - val_acc: 0.0022
Epoch 8/50
112720/112720 [==============================] - 1s - loss: 846542

112720/112720 [==============================] - 3s - loss: -48.4189 - acc: 5.2342e-04 - val_loss: -16.3373 - val_acc: 5.7665e-04
Epoch 3/50
112720/112720 [==============================] - 3s - loss: -54.1044 - acc: 2.3066e-04 - val_loss: -19.5336 - val_acc: 4.4358e-05
Epoch 4/50
112720/112720 [==============================] - 3s - loss: -58.2247 - acc: 8.8715e-06 - val_loss: -23.4423 - val_acc: 0.0000e+00
Epoch 5/50
112720/112720 [==============================] - 3s - loss: -60.9471 - acc: 3.5486e-05 - val_loss: -24.9686 - val_acc: 0.0000e+00
Epoch 6/50
112720/112720 [==============================] - 3s - loss: -62.4613 - acc: 0.0000e+00 - val_loss: -25.7728 - val_acc: 0.0000e+00
Epoch 7/50
112720/112720 [==============================] - 3s - loss: -63.7074 - acc: 8.8715e-06 - val_loss: -26.9110 - val_acc: 0.0000e+00
Epoch 8/50
112720/112720 [==============================] - 3s - loss: -64.6436 - acc: 0.0000e+00 - val_loss: -28.0688 - val_acc: 0.0000e+00
Epoch 9/50
112720/112720

112720/112720 [==============================] - 3s - loss: -38.6422 - acc: 0.0041 - val_loss: -19.8873 - val_acc: 0.0021
Epoch 9/50
112720/112720 [==============================] - 3s - loss: -39.5005 - acc: 0.0039 - val_loss: -21.1852 - val_acc: 0.0019
Epoch 10/50
112720/112720 [==============================] - 3s - loss: -40.1905 - acc: 0.0040 - val_loss: -22.2700 - val_acc: 0.0020
Epoch 11/50
112720/112720 [==============================] - 3s - loss: -40.9232 - acc: 0.0036 - val_loss: -23.5699 - val_acc: 0.0021
Epoch 12/50
112720/112720 [==============================] - 3s - loss: -41.5710 - acc: 0.0037 - val_loss: -24.3385 - val_acc: 0.0021
Epoch 13/50
112720/112720 [==============================] - 3s - loss: -42.1585 - acc: 0.0038 - val_loss: -25.3761 - val_acc: 0.0021
Epoch 14/50
112720/112720 [==============================] - 3s - loss: -42.6143 - acc: 0.0035 - val_loss: -26.0469 - val_acc: 0.0021
Epoch 15/50
112720/112720 [==============================] - 3s - loss: -43

112720/112720 [==============================] - 3s - loss: -37.7251 - acc: 0.0029 - val_loss: 7.6906 - val_acc: 0.0020
Epoch 18/50
112720/112720 [==============================] - 3s - loss: -37.9966 - acc: 0.0030 - val_loss: 7.2442 - val_acc: 0.0021
Epoch 19/50
112720/112720 [==============================] - 3s - loss: -38.3379 - acc: 0.0030 - val_loss: 6.7073 - val_acc: 0.0021
Epoch 20/50
112720/112720 [==============================] - 3s - loss: -38.7277 - acc: 0.0029 - val_loss: 6.3025 - val_acc: 0.0020
Epoch 21/50
112720/112720 [==============================] - 3s - loss: -38.8783 - acc: 0.0027 - val_loss: 5.8062 - val_acc: 0.0020
Epoch 22/50
112720/112720 [==============================] - 3s - loss: -38.9600 - acc: 0.0030 - val_loss: 5.3949 - val_acc: 0.0020
Epoch 23/50
112720/112720 [==============================] - 3s - loss: -39.4003 - acc: 0.0027 - val_loss: 5.0259 - val_acc: 0.0021
Epoch 24/50
112720/112720 [==============================] - 3s - loss: -39.6188 - acc: 

112720/112720 [==============================] - 4s - loss: -21.8636 - acc: 0.0187 - val_loss: 9.3290 - val_acc: 0.0232
Epoch 28/50
112720/112720 [==============================] - 4s - loss: -22.0967 - acc: 0.0182 - val_loss: 9.0291 - val_acc: 0.0233
Epoch 29/50
112720/112720 [==============================] - 4s - loss: -23.6027 - acc: 0.0178 - val_loss: 7.0771 - val_acc: 0.0224
Epoch 30/50
112720/112720 [==============================] - 4s - loss: -24.4747 - acc: 0.0175 - val_loss: 6.8229 - val_acc: 0.0228
Epoch 31/50
112720/112720 [==============================] - 4s - loss: -24.7761 - acc: 0.0168 - val_loss: 6.7719 - val_acc: 0.0231
Epoch 32/50
112720/112720 [==============================] - 4s - loss: -24.9461 - acc: 0.0161 - val_loss: 6.5124 - val_acc: 0.0230
Epoch 33/50
112720/112720 [==============================] - 4s - loss: -25.2402 - acc: 0.0157 - val_loss: 6.3633 - val_acc: 0.0231
Epoch 34/50
112720/112720 [==============================] - 4s - loss: -25.2493 - acc: 

112720/112720 [==============================] - 4s - loss: -41.5655 - acc: 0.0119 - val_loss: -21.4453 - val_acc: 0.0341
Epoch 37/50
112720/112720 [==============================] - 4s - loss: -41.7234 - acc: 0.0119 - val_loss: -21.5505 - val_acc: 0.0341
Epoch 38/50
112720/112720 [==============================] - 4s - loss: -41.7541 - acc: 0.0119 - val_loss: -21.7070 - val_acc: 0.0341
Epoch 39/50
112720/112720 [==============================] - 4s - loss: -41.8912 - acc: 0.0119 - val_loss: -21.8231 - val_acc: 0.0341
Epoch 40/50
112720/112720 [==============================] - 4s - loss: -42.0005 - acc: 0.0119 - val_loss: -21.8839 - val_acc: 0.0341
Epoch 41/50
112720/112720 [==============================] - 4s - loss: -42.1658 - acc: 0.0119 - val_loss: -21.9483 - val_acc: 0.0341
Epoch 42/50
112720/112720 [==============================] - 4s - loss: -42.1984 - acc: 0.0119 - val_loss: -22.0308 - val_acc: 0.0341
Epoch 43/50
112720/112720 [==============================] - 4s - loss: -4

112720/112720 [==============================] - 4s - loss: -44.8904 - acc: 0.0000e+00 - val_loss: -31.7283 - val_acc: 0.0000e+00
Epoch 44/50
112720/112720 [==============================] - 4s - loss: -44.9334 - acc: 0.0000e+00 - val_loss: -31.7739 - val_acc: 0.0000e+00
Epoch 45/50
112720/112720 [==============================] - 4s - loss: -45.0272 - acc: 0.0000e+00 - val_loss: -31.8013 - val_acc: 0.0000e+00
Epoch 46/50
112720/112720 [==============================] - 4s - loss: -45.0394 - acc: 0.0000e+00 - val_loss: -31.8606 - val_acc: 0.0000e+00
Epoch 47/50
112720/112720 [==============================] - 4s - loss: -45.1115 - acc: 0.0000e+00 - val_loss: -31.9563 - val_acc: 0.0000e+00
Epoch 48/50
112720/112720 [==============================] - 4s - loss: -45.1724 - acc: 0.0000e+00 - val_loss: -31.9881 - val_acc: 0.0000e+00
Epoch 49/50
112720/112720 [==============================] - 4s - loss: -45.1839 - acc: 0.0000e+00 - val_loss: -32.0328 - val_acc: 0.0000e+00
Epoch 50/50
22544/

In [ ]:
scores.sort_values("test_score", ascending=False)

In [ ]:
scores.to_pickle("dataset/vae_only_feature_extraction_scores.pkl")